In [2]:
import dgl
import torch
# from Academic_GNN_Module.util import DBLP_PATH

graph_list, _ = dgl.load_graphs(f'../save3/graph_vfc.graph')
graph = graph_list[0]
print(graph)

Using backend: pytorch


Graph(num_nodes={'author': 3306925, 'country': 54, 'field': 142726, 'org': 379, 'paper': 5319925, 'topic': 16081, 'venue': 46347},
      num_edges={('country', 'has', 'org'): 432, ('field', 'has', 'venue'): 29480166, ('org', 'has', 'author'): 7209864, ('paper', 'cites', 'paper'): 25626821, ('paper', 'written', 'author'): 13024047, ('topic', 'contains', 'paper'): 13160952, ('venue', 'publishes', 'paper'): 3278873},
      metagraph=[('country', 'org', 'has'), ('org', 'author', 'has'), ('field', 'venue', 'has'), ('venue', 'paper', 'publishes'), ('paper', 'paper', 'cites'), ('paper', 'author', 'written'), ('topic', 'paper', 'contains')])


In [3]:
g = dgl.node_type_subgraph(graph, ['paper', 'topic'])
print(g)

Graph(num_nodes={'paper': 5319925, 'topic': 16081},
      num_edges={('paper', 'cites', 'paper'): 25626821, ('topic', 'contains', 'paper'): 13160952},
      metagraph=[('paper', 'paper', 'cites'), ('topic', 'paper', 'contains')])


In [4]:
src_top, dst_pap = g.edges(etype='contains')
src_pap1, dst_pap2 = g.edges(etype='cites')

In [5]:
g2 = dgl.heterograph({
    ('paper', 'cites', 'paper'): (src_pap1, dst_pap2), 
    ('topic', 'contains', 'paper'): (src_top, dst_pap), 
    ('paper', 'belong', 'topic'): (dst_pap, src_top)
})
print(g2)

Graph(num_nodes={'paper': 5319925, 'topic': 16081},
      num_edges={('paper', 'belong', 'topic'): 13160952, ('paper', 'cites', 'paper'): 25626821, ('topic', 'contains', 'paper'): 13160952},
      metagraph=[('paper', 'topic', 'belong'), ('paper', 'paper', 'cites'), ('topic', 'paper', 'contains')])


In [6]:
import sys
sys.path.append('../')
from util import myout

In [7]:
cite_year = g.edges['cites'].data['year']
contains_year = g.edges['contains'].data['year']
myout(cite_year)
myout(contains_year)

cite_year : shape=torch.Size([25626821]), tensor([   0,    0,    0,  ..., 2300, 2300, 2300])
contains_year : shape=torch.Size([13160952]), tensor([   0.,    0.,    0.,  ..., 9737., 9737., 9737.], dtype=torch.float64)


In [8]:
g2.edata['year'] = {
    ('paper', 'cites', 'paper'): cite_year, 
    ('topic', 'contains', 'paper'): contains_year, 
    ('paper', 'belong', 'topic'): contains_year
}
print(g2)

Graph(num_nodes={'paper': 5319925, 'topic': 16081},
      num_edges={('paper', 'belong', 'topic'): 13160952, ('paper', 'cites', 'paper'): 25626821, ('topic', 'contains', 'paper'): 13160952},
      metagraph=[('paper', 'topic', 'belong'), ('paper', 'paper', 'cites'), ('topic', 'paper', 'contains')])


In [9]:
year = 2000
# cites_eids = g.filter_edges(lambda x: x.data['year'] == year, etype='cites')
contains_eids = g2.filter_edges(lambda x: x.data['year'] == year, etype='contains')
belong_eids = g2.filter_edges(lambda x: x.data['year'] == year, etype='belong')

year_graph = g2.edge_subgraph({'contains': contains_eids, \
    'belong': belong_eids}, preserve_nodes=True)
print(year_graph)


Graph(num_nodes={'paper': 5319925, 'topic': 16081},
      num_edges={('paper', 'belong', 'topic'): 301320, ('paper', 'cites', 'paper'): 0, ('topic', 'contains', 'paper'): 301320},
      metagraph=[('paper', 'topic', 'belong'), ('paper', 'paper', 'cites'), ('topic', 'paper', 'contains')])


In [10]:
g_topic = dgl.metapath_reachable_graph(year_graph, ['contains', 'belong'])
g_topic = dgl.remove_self_loop(g_topic)
print(g_topic)

Graph(num_nodes=16081, num_edges=1136696,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={})


In [11]:
top1, top2 = g_topic.edges(order='srcdst')
myout(top1)
myout(top2)

top1 : shape=torch.Size([1136696]), tensor([    0,     0,     4,  ..., 16075, 16075, 16075])
top2 : shape=torch.Size([1136696]), tensor([   55,  3242,    13,  ..., 16051, 16065, 16066])


In [12]:
topic_src, paper_dst = year_graph.edges(etype='contains')
myout(topic_src)
myout(paper_dst)

topic_src : shape=torch.Size([301320]), tensor([ 3705,   102, 14446,  ..., 12360,  8363,  8237])
paper_dst : shape=torch.Size([301320]), tensor([652183, 652183, 652184,  ..., 729576, 729576, 729576])


In [24]:
import numpy as np
h_index = torch.zeros(len(top1))
for i in range(len(top1)):
    # id1 = torch.where(topic_src==t1)[0]
    # id2 = torch.where(topic_src==t2)[0]
    pap1 = year_graph.successors(top1[i], etype='contains')
    pap2 = year_graph.successors(top2[i], etype='contains')

    # myout(id1)
    # myout(id2)
    # myout(id2)

    p1, p2 = set(np.array(pap1)), set(np.array(pap2))
    idx = p1 & p2

    h_index[i] = len(idx) if len(idx) > 0 else 0
    myout(h_index[i])
    # myout(idx)

    # myout(p1)


    break


 : shape=torch.Size([]), 1.0


In [20]:
hg = dgl.heterograph({
    ('user', 'follows', 'user'): (torch.tensor([0, 0 , 1, 1]), torch.tensor([1, 2, 3, 4])),
    ('user', 'plays', 'game'): (torch.tensor([3, 4]), torch.tensor([5, 6]))
})
hg.successors(1, etype='follows')

tensor([3, 4])